# LinkedIn Bot to Send REQUEST

In [ ]:

!pip install selenium

!pip install beautifulsoup4

# Download Google Chrome Drive -> https://chromedriver.chromium.org/downloads


In [ ]:
import os, random, sys, time
# from urllib.parse import urlparse
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
browser = webdriver.Chrome('/Users/rishabhkhandelwal/Documents/Automate/Bot/chromedriver')


In [ ]:
browser.get('https://www.linkedin.com/uas/login')

ENTER YOUR USERNAME AND PASSWORD

In [ ]:
elementID = browser.find_element_by_id('username')
elementID.send_keys("ENTER USERNAME")

In [ ]:
elementID = browser.find_element_by_id('password')
elementID.send_keys("ENTER PASSWORD")

In [ ]:
elementID.submit()


COPY YOUR LINKEDIN ID HERE

In [ ]:
visitingProfileID = '/in/CHANGE YOUR ID/'
fullLink = 'https://www.linkedin.com' + visitingProfileID
browser.get(fullLink)

In [ ]:
visitedProfiles = []
profilesQueued = []

In [ ]:
def getNewProfileIDs(soup, profilesQueued):
    profilesID = []
    pav = soup.find('section', {'class': 'pv-profile-section pv-browsemap-section profile-section artdeco-container-card ember-view'})
    all_links = pav.findAll('a', {'class': 'pv-browsemap-section__member ember-view'})
    for link in all_links:
        userID = link.get('href')
        if((userID not in profilesQueued) and (userID not in visitedProfiles)):
            profilesID.append(userID)
    return profilesID

In [ ]:
getNewProfileIDs(BeautifulSoup(browser.page_source), profilesQueued)


In [ ]:
profilesQueued = getNewProfileIDs(BeautifulSoup(browser.page_source), profilesQueued)


In [ ]:
while profilesQueued:
    try:
        visitingProfileID = profilesQueued.pop()
        visitedProfiles.append(visitingProfileID)
        fullLink = 'https://www.linkedin.com' + visitingProfileID
        browser.get(fullLink)

        browser.find_element_by_class_name('pv-s-profile-actions').click()

        browser.find_element_by_class_name('mr1').click()

        customMessage = "Hello, I have found mutual interest area and I would be more than happy to connect with you. Kindly, accept my invitation. Thanks!"
        elementID = browser.find_element_by_id('custom-message')
        elementID.send_keys(customMessage)

        browser.find_element_by_class_name('ml1').click()

        # Add the ID to the visitedUsersFile
        with open('visitedUsers.txt', 'a') as visitedUsersFile:
            visitedUsersFile.write(str(visitingProfileID)+'\n')
        visitedUsersFile.close()

        # Get new profiles ID
        soup = BeautifulSoup(browser.page_source)
        try: 
            profilesQueued.extend(getNewProfileIDs(soup, profilesQueued))
        except:
            print('Continue')

        # Pause
        time.sleep(random.uniform(3, 7)) # Otherwise, sleep to make sure everything loads

        if(len(visitedProfiles)%50==0):
            print('Visited Profiles: ', len(visitedProfiles))

        if(len(profilesQueued)>100000):
            with open('profilesQueued.txt', 'a') as visitedUsersFile:
                visitedUsersFile.write(str(visitingProfileID)+'\n')
            visitedUsersFile.close()
            print('100,000 Done!!!')
            break;
    except:
        print('error')